In [101]:
using LinearAlgebra
using DSP
using Plots
using Statistics
plotlyjs()

Plots.PlotlyJSBackend()

In [6]:
function chirp(f0,f1,N)
    f = LinRange(f0,f1,N)
    phi = (1.0/N).*cumsum(f)
    phi = phi.-phi[1]
    c = sin.((2*pi).*phi)
end

chirp (generic function with 2 methods)

In [235]:
samp_rate = 1250e3
c = chirp(0.0,62.5e3,Int(samp_rate))
xc = xcorr(c,c)
t0 = findmax(xc)[2]

1250000

In [236]:
n1 = randn(length(c))
n1 = sqrt(100).*n1./sqrt((n1'*n1)/(c'*c))
n2 = randn(length(c))
n2 = sqrt(100).*n2./sqrt((n2'*n2)/(c'*c))
Pc = c'*c
Pn = n1'*n1
10*log(10,Pc/Pn) #S/N of -20dB seems plausible for 2MHz bandwidth from link budget
s1 = c+n1
s2 = c+n2
xs = xcorr(s1,s2)
findmax(xs)[2]-t0

-1

In [237]:
SNR_DB = 17
SNR = 10.0^(SNR_DB/10.0)
Nsamp = 1000
ind = zeros(Nsamp)
for k = 1:Nsamp
    n1 = randn(length(c))
    n1 = sqrt(SNR).*n1./sqrt((n1'*n1)/(c'*c))
    n2 = randn(length(c))
    n2 = sqrt(SNR).*n2./sqrt((n2'*n2)/(c'*c))
    ind[k] = findmax(xcorr(c+n1, c+n2))[2]
end
dt = ind.-t0

1000-element Array{Float64,1}:
 -1.0
  1.0
  1.0
 -1.0
 -1.0
 -2.0
  0.0
  0.0
  0.0
  0.0
 -1.0
  0.0
  0.0
  ⋮
 -2.0
 -1.0
 -1.0
  1.0
  1.0
  0.0
  0.0
 -1.0
  1.0
  2.0
  0.0
 -1.0

In [238]:
mean(dt)

-0.009

In [239]:
σ = std(dt)

0.9548983353377634

In [240]:
σt = σ/samp_rate

7.639186682702107e-7

In [241]:
σd = 3e8*σt

229.1756004810632

In [ ]:
#Results (1-σ distance/timing error):
#500 KHz -> 62 meters (2.1e-7 sec)
#625 KHz -> 170 meters (5.67e-7 sec)
#1 MHz -> 216 meters (7.21e-7 sec)
#1.25 MHz -> 226 meters (7.56e-7 sec)
#1.5 MHz -> 229 meters (7.66e-7 sec)
#2 MHz -> 237 meters (7.91e-7 sec)
#4 MHz -> 263 meters (8.79e-7 sec)